In [1]:
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
from sklearn.base     import ClassifierMixin
from sklearn.datasets import load_iris,make_classification
from sklearn.metrics  import *
from sklearn.model_selection import train_test_split

&nbsp;

### Regressão Logistica

Logaritmo das chances : probabilidade de acontecer e não acontecer 

In [ ]:
def logit(p:np.array): 
    return np.log( p/(1-p))

In [ ]:
probalidade = np.around(np.linspace(0.1,0.9,9),2)
print(probalidade)
logit(probalidade)

Função sigmoid: é a inversa do logaritmos das chances

In [ ]:
def sigmoid(z:np.array):
    return 1.0/( 1.0 + np.exp(-1*z) )

In [ ]:
z  = np.linspace(-6,6)
zy = sigmoid(z)

In [ ]:
plt.plot(z,zy,'o-')
plt.axvline(0,color='k')
plt.axhline(0,color='k')
plt.show()

### Função de custo|perda (log-loss)
$$
L(\sigma(z),y;w,b) = -y \cdot \log(\sigma(z) ) - (1-y)\cdot \log(1-\sigma(z))
$$

* Se o y = 0 para uma unica instancia de treinamento a função de perda será igual a $(1-y)\cdot \log(1-h_{\theta}(z))$ , a penalidade aumenta a medida que se aproxima de 1
* Se o y = 1 para uma unica instancia de treinamento a função de perda será igual a $-\log(h_{\theta}(x))$ , a penalidade aumenta a medida que se aproxima de 1

In [ ]:
def perda1(sigmoid): return -np.log(sigmoid)
def perda2(sigmoid): return -np.log(1-sigmoid)

In [ ]:
sigmoidExample = np.linspace(0.001,0.99,100)

In [ ]:
plt.plot(sigmoidExample,perda1(sigmoidExample),label='-log(Ŷ)   se y = 1')
plt.plot(sigmoidExample,perda2(sigmoidExample),label='-log(1-Ŷ) se y = 0')
plt.axhline(0,color='k')
plt.legend(bbox_to_anchor=(1,1,0,0))
plt.show()

In [ ]:
pd.concat([pd.DataFrame({'z':sigmoidExample,'-log(1-h(x))':perda2(sigmoidExample)}),pd.DataFrame({'z':sigmoidExample,'-log(h(x))':perda1(sigmoidExample)})],axis=1)

In [ ]:
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data',header=None,encoding='utf-8')

X = data.iloc[:,[1,2]].values
y = data.iloc[:,4]
y = np.where(y=='Iris-setosa',0,1)
xtrain, xtest, ytrain, ytest = train_test_split(X, y, train_size=100, random_state=0)

&nbsp;

### Algoritmo Regressão Logistica

In [ ]:
class RegressaoLogisticaGD(ClassifierMixin):
    b_ = None # Vies||Bias
    w_ = None # Pesos
    n_ = None # Numero de caracteristicas

    def __init__(self, eta=0.01, max_iter=50, random_state=1,alpha=0):
        self.eta = eta
        self.max_iter = max_iter
        self.random_state = random_state
        self.alpha = alpha
    
    def fit(self, X, y):
        self.n_ = np.size(X, axis=1)
        rgen    = np.random.RandomState(self.random_state)
        self.w_ = rgen.normal( loc=0.0, scale=0.01, size=self.n_ )
        self.b_ = np.float64(0.0)

        self.losses_ = [] #F(x) para cada epoca

        for iter in range(self.max_iter):
            somaPonderada = self.somaPonderada(X)
            valorPrevisto = self.activacaoSigmoid(somaPonderada)
            erros = (y-valorPrevisto)

            self.w_ += self.eta * 2.0 * X.T.dot(erros)/np.size(X, axis=0) + (self.alpha*(self.w_))/np.size(X, axis=0)
            self.b_ += self.eta * 2.0 * np.mean(erros)

            loss = -y.dot(np.log(valorPrevisto)) - ((1-y).dot(np.log(1-valorPrevisto)))/np.size(X, axis=0) + (self.alpha*np.linalg.norm(self.w_))/np.size(X, axis=0)
            self.losses_.append(loss)
        return self

    def somaPonderada(self, X):
        return np.dot(X, self.w_) + self.b_
    
    def activacaoSigmoid(self, Z):
        return 1./(1. + np.exp(-Z))
    
    def predict(self, X):
        return np.where( self.activacaoSigmoid(self.somaPonderada(X))>=0.5,1,0)          

In [ ]:
logitic = RegressaoLogisticaGD(max_iter=100,alpha=1)
logitic.fit(xtrain,ytrain)

In [ ]:
logitic.w_,logitic.b_

In [ ]:
predL = logitic.predict(xtest)

In [ ]:
print(classification_report(ytest,predL))
print(accuracy_score(ytest,predL))

In [ ]:
from plots.PlotRegiaDeDecisao import plotDecisionRegion,plotDecisionRegion2

plotDecisionRegion(xtest,ytest,logitic)

&nbsp;

### Soft Max

In [ ]:
class SoftMaxGD(ClassifierMixin):
    b_ = None # Vies||Bias
    w_ = None # Pesos
    n_ = None # Numero de caracteristicas
    k_ = None
    def __init__(self, eta=0.1, max_iter=50, random_state=1):
        self.eta = eta
        self.max_iter = max_iter
        self.random_state = random_state
    
    def fit(self, X, y):
        y += 1
        self.k_ = len(np.unique(y))

        self.n_ = np.size(X, axis=1)
        rgen    = np.random.RandomState(self.random_state)
        self.w_ = rgen.normal( loc=0.0, scale=0.1, size=(self.k_,self.n_))
        self.b_ = np.zeros(shape=(self.k_,1))
        self.losses_ = []
        
        for iter in range(self.max_iter):
           
            erros = []
            for ki in range(self.k_):
                error = (np.where( y == ki+1,ki+1,0 ) - self.ativacaoSoftMax(X,ki)[0])
                erros.append(error)
            
            erros = np.array(erros).T
           
            for ki in range(self.k_):
                self.w_[ki,:] += self.eta * 2.0 * X.T.dot(erros[:,ki])/np.size(X, axis=0)
                self.b_[ki,:] += self.eta * 2.0 * np.mean(erros[:,ki])
            
            error = 0
            for ki in range(self.k_):
                error += np.sum(np.log(np.where( y == ki+1,ki+1,0 )*self.ativacaoSoftMax(X,ki)[0]))
            

        return self

    def somaSoftMax(self, X, ki):
        return np.dot(X, self.w_[ki,:]) + self.b_[ki,:]
        
    def ativacaoSoftMax(self, X, ki):
        n =  np.size(X, axis=0)
        exp1 = np.around(np.exp(self.somaSoftMax(X,ki)),decimals=5)
        exp2 = np.zeros(shape=(1,n))
       
        for ki in range(self.k_):
            exp2+= np.around(np.exp(self.somaSoftMax(X,ki)),decimals=5)

        

        return exp1/exp2

    def predict(self, X):
        soft = []
        for ki in range(self.k_):
            soft.append(self.ativacaoSoftMax(X,ki)[0])
        soft = np.array(soft).T
        return np.array([np.argmax(xi)for xi in soft])

In [ ]:
X, y = load_iris(return_X_y=True)
X = X[:,[2,3]]
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=0,stratify=y)


In [ ]:
from sklearn.preprocessing import StandardScaler
xtrain = StandardScaler().fit_transform(xtrain)
xtest  = StandardScaler().fit_transform(xtest)

In [ ]:
soft = SoftMaxGD(eta=0.01,max_iter=1000,random_state=1)
soft.fit(xtrain,ytrain)

In [ ]:
np.bincount(ytest)

In [ ]:
soft.w_

In [ ]:
pred = soft.predict(xtest)

In [ ]:
print(classification_report(ytest+1,pred+1))
print(accuracy_score(ytest+1,pred+1))

In [ ]:
confusion_matrix(ytest+1,pred+1)

In [ ]:
from plots.PlotRegiaDeDecisao import plotDecisionRegion,plotDecisionRegion2

plotDecisionRegion2(xtest,ytest,soft)

&nbsp;

### Logistica OVR

In [ ]:
class RegressaoLogisticaOvr(ClassifierMixin):
    def __init__(self, max_iter=100, random_state=None):
        self.max_iter = max_iter
        self.random_state = random_state
        self.listEstimator = None

    def fit(self, X, y):
        labels = np.unique(y)
        self.listEstimator = {}
        for classi in labels:
            y_bin = np.where(y == classi, 1, 0)
            estimator = RegressaoLogisticaGD(max_iter=self.max_iter, random_state=self.random_state)
            estimator.fit(X, y_bin)
            self.listEstimator[classi] = estimator
        return self

    def predict(self, X):
        if self.listEstimator is None:
            raise TypeError('None')
        
        pred = []
        for xi in X:
            class_scores = {}
            for key, estimator in self.listEstimator.items():
                class_scores[key] = estimator.predict(xi.reshape(1, -1)).item()
            pred.append(max(class_scores, key=class_scores.get))
        return np.array(pred)


In [ ]:
X, y = load_iris(return_X_y=True)
X = X[:,[0,1]]
X = ().fit_transform(X)
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=0,stratify=y)

In [ ]:
reg = RegressaoLogisticaOvr(max_iter=1000,random_state=10)
reg.fit(xtrain,ytrain)

In [ ]:
pred = reg.predict(xtest)
pred

In [ ]:
reg.listEstimator[0].w_,reg.listEstimator[1].w_,reg.listEstimator[2].w_

In [ ]:
print(classification_report(ytest,pred))
print(accuracy_score(ytest,pred))

In [ ]:
plotDecisionRegion2(xtest,ytest,reg)